> Model Inference

# 1 - Import Libraries

In [1]:
# Import Libraries
import pickle
import json
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# 2 - Load Files

In [3]:
# Load Files
with open('imputer_pipe.pkl', 'rb') as file_1:
    imputer_pipe = pickle.load(file_1)

with open('final_pipe.pkl', 'rb') as file_2:
    final_pipe = pickle.load(file_2)

with open('col.txt', 'r') as file_3:
    col = json.load(file_3)

with open('selected_col.txt', 'r') as file_4:
    selected = json.load(file_4)

with open('data_sample.txt', 'r') as file_5:
    data_sample = json.load(file_5)

# Convert JSON file to Pandas Dataframe
data_inference = pd.read_json(data_sample)
data_test = data_inference.head(1)
data_test


,user_id,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
4137,a3916cd4d3394fdd,60,F,City,Basic Membership,2015-10-12,No,Credit/Debit Card Offers,Desktop,Fiber_Optic,...,388.39,15632.04,26,0.0,No,Yes,Yes,Unsolved,Too many ads,1


In [4]:
# Load Model
model_ann = load_model('sequential_model.h5')

# 3 - Model Infrencing

In [9]:
# Imputer Pipe
data_imputed = imputer_pipe.transform(data_test)

In [10]:
# Imputing Based on Imported Columns
df_imputed = pd.DataFrame(data_imputed, columns=col)

In [11]:
# Drop Features Based on Imported Features
df_selected = df_imputed[selected]

In [12]:
# Show Results
df_selected

,age,days_since_last_login,avg_time_spent,avg_transaction_value,gender,internet_option,used_special_discount,past_complaint,complaint_status
0,60.0,15.0,388.39,15632.04,F,Fiber_Optic,No,Yes,Unsolved


In [13]:
# Scaler & Encoder
data_final = final_pipe.transform(df_selected)

In [14]:
# Predict Using Artificial Neural Network
y_pred_inf = model_ann.predict(data_final)
y_pred_inf = np.where(y_pred_inf>=0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 0s 234ms/step


array([[1]])

In [16]:
data_test['churn_prediction'] = y_pred_inf
data_test[['churn_risk_score', 'churn_prediction']]

<ipython-input-16-4f1af87361d6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['churn_prediction'] = y_pred_inf


,churn_risk_score,churn_prediction
4137,1,1
